<a href="https://colab.research.google.com/github/QUANT-CU/.github/blob/main/VIX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance

In [3]:
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import timedelta

tk = yf.Ticker("^SPX")

exps = tk.options

options = pd.DataFrame()
for e in exps:
    opt = tk.option_chain(e)
    opt.calls['optionType'] = 'Call'
    opt.puts['optionType'] = 'Put'
    opt.calls['expirationDate'] = e
    opt.puts['expirationDate'] = e

    options = pd.concat([options, pd.concat([opt.calls, opt.puts])])
options = options.reset_index(drop=True)
options['midPrice'] = (options['bid'] + options['ask']) / 2
options['expirationDate'] = pd.to_datetime(options['expirationDate'])


t0 = pd.to_datetime(opt.underlying['regularMarketTime'] * 1000, unit='ms').normalize()


tau = 30

t0plustau = t0 + timedelta(tau)


wfri = pd.date_range(t0, periods=10, freq='W-FRI')

tenor1_idx = np.where(t0plustau > wfri)[0][-1]
tenor2_idx = np.where(t0plustau < wfri)[0][0]

tenor1 = wfri[tenor1_idx]
tenor2 = wfri[tenor2_idx]

print(tenor1, tenor2)

S0 = opt.underlying['regularMarketPrice']


tau1 = (tenor1 - t0).days / 365
tau2 = (tenor2 - t0).days / 365

r = 0.03
F0_tau1 = S0 * np.exp(r * tau1)
F0_tau2 = S0 * np.exp(r * tau2)

print(S0, F0_tau1, F0_tau2)


def vol_from_opt_price(options, tenor ,tau, F0):
    opts = options[options['expirationDate'] == tenor]
    puts = opts[opts['optionType'] == 'Put']
    puts = puts[puts['strike'] < F0]

    calls = opts[opts['optionType'] == 'Call']

    kgeqf = np.where(calls['strike'] >= F0)[0]
    calls = calls.iloc[np.append(kgeqf[0] - 1, kgeqf)]

    Ki = puts['strike']
    Qi = puts['midPrice']

    putsidevol = (Qi * (1 / Ki - 1 / Ki.shift(-1))).sum()
    Ki = calls['strike']
    Qi = calls['midPrice']

    callsidevol = (Qi * (1 / Ki - 1 / Ki.shift(-1))).sum()

    return 2 / tau * np.exp(r * tau) * (putsidevol + callsidevol) - 1 / tau * (F0 / calls.iloc[0].strike - 1) ** 2

Near_vol = vol_from_opt_price(options, tenor1, tau1, F0_tau1)
Nextterm_vol = vol_from_opt_price(options, tenor2, tau2, F0_tau2)

tweight1 = (tenor2 - t0plustau).days / (tenor2 - tenor1).days
tweight2 = (t0plustau - tenor1).days / (tenor2 - tenor1).days

vix = np.sqrt((tau1 * Near_vol * tweight1 + tau2 * Nextterm_vol * tweight2) * 365 / tau) * 100

realvix = yf.Ticker("^VIX")
data = realvix.history(period="1d")

print(f"The VIX calculated from market option data: {vix:.2f}")
print(f"The VIX from Yahoo Finance: {data['Close'].iloc[-1]:.2f}")

2024-11-22 00:00:00 2024-11-29 00:00:00
5827.54 5839.526708153887 5842.887402531381
The VIX calculated from market option data: 19.63
The VIX from Yahoo Finance: 19.63
